# TensorFlow・Kerasのトレーニング♨

## CNN - 初歩編＋転移学習・ファインチューニング 編
- 初歩編の10値分類を転移学習・ファインチューニングで試す。
- このNotebookを実行するにはGPU搭載のPC上で実行する必要がある。

## [目次](TableOfContents.ipynb)

## 参考
開発基盤部会 Wiki
- データマイニング（DM）- Python - DL  
https://dotnetdevelopmentinfrastructure.osscons.jp/index.php?%E3%83%87%E3%83%BC%E3%82%BF%E3%83%9E%E3%82%A4%E3%83%8B%E3%83%B3%E3%82%B0%EF%BC%88DM%EF%BC%89-%20Python%20-%20DL

## [環境準備](TensorFlowAndKeras0.ipynb)

### インポート

In [ ]:
import io
import requests

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn import datasets
from sklearn import metrics
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

import tensorflow as tf
from tensorflow.keras.layers import BatchNormalization
print(tf.__version__)

import keras
print(keras.__version__)
# モデル定義
from keras.models import Model, Sequential, model_from_json, load_model
from keras.layers import Dense, Input, Activation, Flatten, Dropout, LSTM
from keras.layers import Conv2D, MaxPool2D
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras import optimizers
from keras.optimizers import SGD, Adam
# その他
from keras.applications.vgg16 import VGG16
from tensorflow.keras.utils import to_categorical

import cv2

import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

### 共通関数

#### 画像の前処理（様々な変換処理）

##### 変換処理

In [ ]:
# 画像比較
def diff_image_info(img1, img2, cmap1=None, cmap2=None):
    print(img1.shape)
    print(img2.shape)
    plt.subplot(1, 2, 1)
    plt.imshow(img1, cmap1)
    plt.subplot(1, 2, 2)
    plt.imshow(img2, cmap2)

# 画像回転
def opencv_rotate(img, angle=30):
    size = (img.shape[0], img.shape[1])
    center = (int(size[0]/2), int(size[1]/2))
    rotation_matrix = cv2.getRotationMatrix2D(center, angle, 1.0)
    return cv2.warpAffine(img, rotation_matrix, size)

# 画像並進
def opencv_move(img, h=100, v=50):
    rows, cols, channnels = img.shape
    M = np.float32([[1,0,h],[0,1,v]])
    return cv2.warpAffine(img, M, (cols, rows))

# 画像拡大
def opencv_zoomin(img, h=2.0, v=2.0):
    zoomed = cv2.resize(img, None, fx=h, fy=v)
    height_1, width_1, channel_1 = img.shape
    height_2, width_2, channel_2 = zoomed.shape
    x =  int((width_2 - width_1) / 2)
    y =  int((height_2 - height_1) / 2)
    return zoomed[y:y+height_1, x:x+width_1]

# ヒストグラム平坦化
def opencv_clahe(img, clipLimit=2.0, tileGridSize=(8,8)):
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
    return clahe.apply(img)

# Mean Subtraction
def img_meansubtraction(img):
    img2 = img.astype('f')
    
    # スケーリング処理
    img2 /= 255 # 0～1正規化
    img2 -= np.mean(img2) # Mean Subtraction
    return img2

# 画像のピクセル値の正規化
def img_min_max(img):
    img_min = img.min()
    img_max = img.max()
    return (img - img_min) / (img_max - img_min)

# ガンマ変換
def opencv_gamma(img, gamma=0.5):
    look_up_table = np.zeros((256, 1), dtype='uint8')
    for i in range(256):
        look_up_table[i][0] = 255 * pow(float(i) / 255, 1.0 / gamma)
    return cv2.LUT(img, look_up_table)

# ガウシアンノイズ
def img_gaussian_noise(img, loc=0.0, scale=5.0):
    row, col, ch = img.shape
    noise = np.random.normal(loc,scale,(row,col,ch))
    noise = noise.reshape(row,col,ch)
    noised = img + noise
    noised /= 255
    return noised

# リサイズ
def opencv_resize(img, h, w):
    return cv2.resize(img, (h, w))

# クロップ
def img_cropping(img, h, w):
    return img[h[0]:h[1], w[0]:w[1], :]

# クロップ（％
def img_cropping_p(img, hp, wp):
    h, w, c = img.shape
    return img[int(h * hp[0]): int(h * hp[1]):, int(w * wp[0]): int(w * wp[1]), :]

# bgr -> rgb
def opencv_bgr2rgb(img):
    return cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

# グレースケール化
def opencv_rgb2gray(img):
    return cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)

# 2値化
def opencv_binary(img, threshold=125):
    # 二値化(閾値を超えた画素を255にする。)
    return cv2.threshold(grayed, threshold, 255, cv2.THRESH_BINARY)

# 左右反転
def opencv_flip_horizontal(img):
    return cv2.flip(img, 1)

# 上下反転
def opencv_flip_vertical(img):
    return cv2.flip(img, 0)

# 平滑化
def opencv_gaussian_blur(img, ksize=(13, 13), sigma=0):
    return cv2.GaussianBlur(img, ksize, sigma)

##### バッチ変換とバッチ拡張
- get_changedとget_augmentedは別々に定義する。
- get_augmentedはtrainに対してのみ実行してtestに対しては実行しないので。

###### バッチ変換
get_changed

In [ ]:
def get_changed(img):
    # グレースケール化
    img = opencv_rgb2gray(img)
    
    # ヒストグラム平坦化
    img = opencv_clahe(img, tileGridSize=(4,4))
    
    # 平滑化
    img = opencv_gaussian_blur(img, ksize=(3, 3))
    
    # カラーでなくなっている場合、
    # チャンネルの次元が減っているので、元に戻す。
    return img # img[:, :, np.newaxis]

###### バッチ拡張
get_augmented

In [ ]:
def get_augmented(img):
    
    # 左右反転
    if np.random.rand() > 0.5:
        img = opencv_flip_horizontal(img)
        
    # 左右度回転
    if np.random.rand() > 0.5:
         # -45 ～ +45 の範囲で
        angle = np.random.randint(-45, 45)
        rotation_matrix = opencv_rotate(img, angle)
        img = cv2.warpAffine(img, rotation_matrix, size)
        
    return img

#### 画像確認

##### 画像とラベルの確認

In [ ]:
def show_image_info(x, y, label, index):
    print("label: ", label[y[index]])
    print("Image: ")
    plt.imshow(x[index].astype(np.uint8))
    plt.show()

##### ランダムな画像一覧

In [ ]:
def show_image_list(x, y, numOfCls=10, numOfImgInCls=10):
    pos = 1
    plt.figure(figsize=(numOfCls, numOfImgInCls))

    # クラス毎に以下の処理を繰り返す。
    for targetClass in range(numOfCls):
        targetIdx = []
        
        # 当該クラスの画像のインデックスリストを取得
        for i in range(len(y)):
            if y[i] == targetClass:
                targetIdx.append(i)
        
        # 当該クラスのインデックスリストからランダムに選んだ最初のn個の画像を描画
        np.random.shuffle(targetIdx)
        for idx in targetIdx[:numOfImgInCls]:
            plt.subplot(numOfCls, numOfImgInCls, pos)
            plt.imshow(x[idx])
            plt.axis('off')
            pos += 1

plt.show()

##### 誤った推論の画像を表示する関数

In [ ]:
def show_incorrect_image_list(x, y, y_pred, label, numOfImg=10):
    index = (y != y_pred)
    for i, val in enumerate(index):
        if val == True:
            print('predict: ', label[y_pred[i]])
            print('answer : ', label[y[i]])
            show_image_info(x, y, label, i)
            numOfImg -= 1
            if numOfImg <= 0:
                break

#### 分類問題関連

##### [分類問題のメトリック表示関数](ScikitLearnTraining5.ipynb)

In [ ]:
def print_metrics(label, pred):
    print('accuracy: %.3f' % metrics.accuracy_score(label, pred)) # 正答率
    
    print('\nmicro') # ミクロ平均
    print('recall: %.3f' % metrics.recall_score(label, pred, average='micro')) # 再現率
    print('precision: %.3f' % metrics.precision_score(label, pred, average='micro')) # 適合率
    print('f1_score: %.3f' % metrics.f1_score(label, pred, average='micro')) # f値
    
    print('\nmacro') # マクロ平均
    print('recall: %.3f' % metrics.recall_score(label, pred, average='macro')) # 再現率
    print('precision: %.3f' % metrics.precision_score(label, pred, average='macro')) # 適合率
    print('f1_score: %.3f' % metrics.f1_score(label, pred, average='macro')) # f値

##### 混同行列のグラフ化関数

In [ ]:
def plot_cm(confmat, label):
    numOfCls = len(label)
    fig, ax = plt.subplots(figsize=(numOfCls, numOfCls))
    ax.matshow(confmat, cmap=plt.cm.Blues, alpha=0.3)
    for i in range(confmat.shape[0]):
        for j in range(confmat.shape[1]):
            ax.text(x=j, y=i, s=confmat[i, j], va='center', ha='center')
            
    # 軸目盛を打つ場所を決める
    ax.set_xticks(np.arange(len(label)))
    ax.set_yticks(np.arange(len(label)))
    # 軸目盛を設定
    ax.set_xticklabels(label)
    ax.set_yticklabels(label)
    #plt.xticks(np.array(label)) # x軸の目盛りを指定
    #plt.yticks(np.array(label)) # y軸の目盛りを指定
    
    plt.xlabel('y_pred label')
    plt.ylabel('y label')
    plt.show()

#### 学習履歴表示関数

##### 損失

In [ ]:
def plot_history_loss(hist):
    plt.plot(hist.history['loss'],label="loss for training")
    plt.plot(hist.history['val_loss'],label="loss for validation")
    plt.title('model loss')
    plt.xlabel('epoch')
    plt.ylabel('loss')
    plt.legend(loc='best')
    plt.show()

##### 正解率

In [ ]:
def plot_history_acc(hist):
    plt.plot(hist.history['accuracy'],label="accuracy for training")
    plt.plot(hist.history['val_accuracy'],label="accuracy for validation")
    plt.title('model accuracy')
    plt.xlabel('epoch')
    plt.ylabel('accuracy')
    plt.legend(loc='best')
    plt.show()

## 転移学習・ファインチューニング

### データ

#### 生成

In [ ]:
from keras.datasets import cifar10
(x_train_org, y_train_org), (x_test_org, y_test_org) = cifar10.load_data()
print(x_train_org.shape, x_test_org.shape)
print(y_train_org.shape, y_test_org.shape)

##### 加工
一旦、結合します。

In [ ]:
x_org = np.vstack([x_train_org, x_test_org])
y_org = np.concatenate([y_train_org, y_test_org])
print(x_org.shape, y_org.shape)

##### 理解

In [ ]:
show_image_info(x_org, y_org.flatten(), [0,1,2,3,4,5,6,7,8,9], 10)

In [ ]:
show_image_list(x_org, y_org.flatten(), 10, 10)

##### 準備

###### 前処理関数定義

In [ ]:
def preprocess_vgg16(img, size=(32, 32)):
    
    # リサイズ
    img= cv2.resize(img, size)
    
    # RGBチャネルからそれぞれvgg指定の値を引く
    # (mean-subtractionに相当)
    img[:, :, 0] -= 103.939
    img[:, :, 1] -= 116.779
    img[:, :, 2] -= 123.68
    
    return img

###### XのKeras入力用型変換

In [ ]:
x_org = x_org.astype('f')

###### 正解ラベルのOne-Hotエンコーディング
Kerasでは正解ラベルはOne-Hotベクトル化が必要。

In [ ]:
# エンコーディング
y_encded = to_categorical(y_org.flatten(), num_classes=10).astype('i') 
# デコーディング
print((y_encded.argmax(axis=1) == y_org.flatten()).all())

###### 学習・テストデータの分割

In [ ]:
x_train = x_org[:50000]
x_test = x_org[50000:]
y_train = y_encded[:50000]
y_test = y_encded[50000:]
print(x_train.shape, x_test.shape)
print(y_train.shape, y_test.shape)

###### 画像の前処理（様々な変換処理）
leakageの防止（データ分割後にデータ変換・拡張等）

In [ ]:
# 必要に応じて、
# get_changed(img)でデータ変換し、
# get_augmented(img)でデータ拡張する。

x_train_list = []
for img in x_train:
    x_train_list.append(preprocess_vgg16(img))
x_train_aug = np.array(x_train_list)

x_test_list = []
for img in x_test:
    x_test_list.append(preprocess_vgg16(img))
x_test_aug = np.array(x_test_list)

### モデリング

#### CNNの定義

##### inputs
全結合層部分を含まない学習済みモデルの読込
- weights ：重みの初期値（'random': ランダム, 'imagenet': 学習済みの重み）
- include_top ：全結合層のダウンロード（True: する, False: しない）
- input_tensor：入力テンソルの型（縦, 横, チャンネル）

In [ ]:
from tensorflow.keras.layers import Input
from tensorflow.keras.applications.vgg16 import VGG16
base_model = VGG16(weights='imagenet', include_top=False, input_tensor=Input(shape=(32, 32, 3)))
print(base_model.output_shape)

##### output
追加する全結合層部分を定義

In [ ]:
n_class = 10
top_model = Sequential()
top_model.add(Flatten(input_shape=base_model.output_shape[1:]))
top_model.add(Dense(256))
top_model.add(Activation('relu'))
top_model.add(Dropout(0.5))
top_model.add(Dense(n_class))
top_model.add(Activation('softmax'))

##### 追加結合
全結合層部分を含まない学習済みモデルに全結合層部分を追加

In [ ]:
model = Model(inputs=base_model.input, outputs=top_model(base_model.output))

##### コンパイル
畳み込み層のパラメタを固定してコンパイル

In [ ]:
for layer in model.layers[:15]:
    layer.trainable = False
    
model.compile(loss='categorical_crossentropy',
              optimizer=SGD(lr=0.0001),
              metrics=['accuracy'])

#### 確認

In [ ]:
model.summary()

### 実行

#### 学習
- VGG16はCPUで扱うには大規模過ぎる。
- CPUで扱う場合は、epoch数を1に設定。

In [ ]:
batch_size = 100
n_epoch = 1 #30 # 3000 # 試用なので回数を減らす
hist = model.fit(x_train_aug, y_train,
                 validation_data=(x_test_aug, y_test),
                 batch_size=batch_size,
                 epochs=n_epoch,
                 verbose=1)

In [ ]:
score = model.evaluate(x_test_aug, y_test)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

In [ ]:
plot_history_loss(hist)

In [ ]:
plot_history_acc(hist)